In [155]:
import pandas as pd
import seaborn as sns
import numpy as np

In [156]:
df = sns.load_dataset('Titanic')

In [157]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [158]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [159]:
x = df.drop(columns='survived')
y = df['survived']

In [160]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [161]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [162]:

def dropping_cols(x,y=None) :
    cols = ['survived','sibsp', 'parch', 'class', 'who', 'deck', 'embark_town',
       'alive', 'alone' ]
    existing_cols = [c for c in cols if c in x_train.columns]
    
    return x.drop(columns = existing_cols, axis=1),y

In [193]:
a,b = dropping_cols(x_train,y_train)
print(a)
print(b)

     pclass     sex   age      fare embarked  adult_male
445       1    male   4.0   81.8583        S       False
650       3    male   NaN    7.8958        S        True
172       3  female   1.0   11.1333        S       False
450       2    male  36.0   27.7500        S        True
314       2    male  43.0   26.2500        S        True
..      ...     ...   ...       ...      ...         ...
106       3  female  21.0    7.6500        S       False
270       1    male   NaN   31.0000        S        True
860       3    male  41.0   14.1083        S        True
435       1  female  14.0  120.0000        S       False
102       1    male  21.0   77.2875        S        True

[623 rows x 6 columns]
445    1
650    0
172    1
450    0
314    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: survived, Length: 623, dtype: int64


In [163]:
def filling_nans(x,y=None):
    x['age'] = x['age'].fillna(x['age'].dropna().mean())
    
    
    x['embarked'] = x['embarked'].fillna(x['embarked'].dropna().mode())
    
    return x,y

In [164]:
def encodding(x,y=None):
    
    x_encdoded =  pd.get_dummies(x, drop_first= True)
    return x_encdoded.astype('float64'),y

In [165]:
# encodding(filling_nans(dropping_cols(df)))

In [166]:
pipeline = Pipeline([
    ('Dropping Columns', FunctionTransformer(dropping_cols) ),
    ('Filling Nulls', FunctionTransformer(filling_nans) ),
    ('Encoding ', FunctionTransformer(encodding)),
    ('Scaling', StandardScaler()),
    ('Clf', RandomForestClassifier())
])

In [167]:
# pipeline.fit(x_train,y_train)

In [168]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [201]:
preprocessing = ColumnTransformer(transformers= [
    ('Age' , SimpleImputer(strategy= 'mean'), ['age']),
    ('Gender', OneHotEncoder(handle_unknown='ignore'),['sex']),
    ('Embarked', Pipeline([
        ('Impute', SimpleImputer(strategy= 'most_frequent')),
        ('Encoding', OneHotEncoder( handle_unknown='ignore'))]),['embarked'])
    ], remainder = 'passthrough')

In [202]:
pipeline = Pipeline([
    ('Dropining Unwanted Cols',FunctionTransformer(dropping_cols)),
    ('Preprocessing', preprocessing),
    ('Scale', StandardScaler()),
    ('Clf', RandomForestClassifier(n_estimators=100))
])

In [ ]:
# pipeline.fit(x_train,y_train)

ValueError: Expected 2D array, got 1D array instead:
array=[     pclass     sex   age      fare embarked  adult_male
 445       1    male   4.0   81.8583        S       False
 650       3    male   NaN    7.8958        S        True
 172       3  female   1.0   11.1333        S       False
 450       2    male  36.0   27.7500        S        True
 314       2    male  43.0   26.2500        S        True
 ..      ...     ...   ...       ...      ...         ...
 106       3  female  21.0    7.6500        S       False
 270       1    male   NaN   31.0000        S        True
 860       3    male  41.0   14.1083        S        True
 435       1  female  14.0  120.0000        S       False
 102       1    male  21.0   77.2875        S        True

 [623 rows x 6 columns]                                   None].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.